# USE THE NEW DATA !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [1]:
import os
import time 

import ais
import matplotlib.pyplot as plt
from priors import NormalPrior
from kernels import ParsenDensityEstimator
from scipy.stats import norm

import tensorflow as tf
import numpy as np
import mnist_data
import vae
import plot_utils
import glob
import sys
import time
import scipy
from sklearn.decomposition import PCA
from skimage.feature import hog

""" parameters """

# source activate tensorflow_p36 && pip install pillow && pip install scikit-image && pip install scikit-learn
# source activate tensorflow_p36 && python run_main.py --dim_z 10 --num_epochs 300
# source activate tensorflow_p36 && python mc_attack_cvae.py 299 5 && python mc_attack_cvae.py 299 5 && sudo shutdown -P now

# combined:
# source activate tensorflow_p36 && pip install pillow && pip install scikit-image && pip install scikit-learn && python run_main.py --dim_z 10 --num_epochs 300 && python mc_attack_cvae.py 299 5 && python mc_attack_cvae.py 299 5 && sudo shutdown -P now
# source activate tensorflow_p36 && python mc_attack_cvae.py 299 5 && python mc_attack_cvae.py 299 5 && sudo shutdown -P now
# source activate tensorflow_p36 && python mc_attack_cvae.py 299 5 && python mc_attack_cvae.py 299 5 && sudo shutdown -P now
model_no = '299' # which model to attack
exp_nos = int(1) # how many different experiments ofr specific indexes

instance_no = np.random.randint(10000)
experiment = 'MC_ATTACK_CVAE' + str(instance_no)
percentage = 0.1

dt = np.dtype([('instance_no', int),
               ('exp_no', int),
               ('method', int), # 1 = white box, 2 = euclidean_PCA, 3 = hog, 4 = euclidean_PCA category, 5 = hog category, 6 = ais
               ('pca_n', int),
               ('percentage_of_data', float),
               ('percentile', float),
               ('mc_euclidean_no_batches', int), # stuff
               ('mc_hog_no_batches', int), # stuff
               ('sigma_ais', float),
               ('11_perc_mc_attack_log', float),
               ('11_perc_mc_attack_eps', float),
               ('11_perc_mc_attack_frac', float), 
               ('50_perc_mc_attack_log', float), 
               ('50_perc_mc_attack_eps', float),
               ('50_perc_mc_attack_frac', float),
               ('50_perc_white_box', float),
               ('11_perc_white_box', float),
               ('50_perc_ais', float),
               ('50_perc_ais_acc_rate', float),
              ])

experiment_results = []

IMAGE_SIZE_MNIST = 28
n_hidden = 500
dim_img = IMAGE_SIZE_MNIST**2  # number of pixels for a MNIST image
dim_z = 10
model_no = '299'

""" prepare MNIST data """

train_total_data, train_size, valid_total_data, validation_size, test_total_data, test_size, _, _ = mnist_data.prepare_MNIST_data(reuse=True)
# compatibility with old attack
vaY = np.where(valid_total_data[:,784:795] == 1)[1]
trY = np.where(train_total_data[:,784:795] == 1)[1]
teY = np.where(test_total_data[:,784:795] == 1)[1]
vaX = valid_total_data[:,0:784]
trX = train_total_data[:,0:784]
teX = test_total_data[:,0:784]
n_samples = train_size

""" build graph """

# input placeholders
# In denoising-autoencoder, x_hat == x + noise, otherwise x_hat == x
x_hat = tf.placeholder(tf.float32, shape=[None, dim_img], name='input_img')
x = tf.placeholder(tf.float32, shape=[None, dim_img], name='target_img')
y = tf.placeholder(tf.float32, shape=[None, mnist_data.NUM_LABELS], name='target_labels')

# dropout
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

# input for PMLR
z_in = tf.placeholder(tf.float32, shape=[None, dim_z], name='latent_variable')
fack_id_in = tf.placeholder(tf.float32, shape=[None, mnist_data.NUM_LABELS], name='latent_variable')

# network architecture
x_, z, loss, neg_marginal_likelihood, KL_divergence = vae.autoencoder(x_hat, x, y, dim_img, dim_z, n_hidden, keep_prob)

decoded = vae.decoder(z_in, fack_id_in, dim_img, n_hidden)

sess = tf.InteractiveSession()

saver = tf.train.Saver()
saver = tf.train.import_meta_graph('models/mnist_gan.ckpt-'+model_no+'.meta')
saver.restore(sess, './models/mnist_gan.ckpt-'+model_no)

c:\users\d065042\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting data\train-images-idx3-ubyte.gz
Extracting data\train-labels-idx1-ubyte.gz
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from ./models/mnist_gan.ckpt-299


In [4]:
import tensorflow as tf
import numpy as np
import mnist_data
import os
import vae
import plot_utils
import glob
import sys
import time
import scipy
from sklearn.decomposition import PCA
from skimage.feature import hog

def OneHot(X, n=10, negative_class=0.):
    X = np.asarray(X).flatten()
    if n is None:
        n = np.max(X) + 1
    Xoh = np.ones((len(X), n)) * negative_class
    Xoh[np.arange(len(X)), X] = 1.
    return Xoh

# indexes 1,11,21,31,... are ones, 2,12,22 are twos etc.
def generate_samples_for_digits(sample_size=100):
    
    Z_np_sample_buffer = np.random.randn(sample_size, dim_z)
    
    digits = np.zeros((sample_size,)).astype(int)
    for i in range(len(digits)):
        digits[i] = i%10
    Y_np_sample = OneHot( digits)

    generated_samples = sess.run(decoded, feed_dict={z_in: Z_np_sample_buffer, fack_id_in: Y_np_sample, keep_prob : 1})

    if (np.any(np.isnan(generated_samples))) or (not np.all(np.isfinite(generated_samples))):
        print('Problem')
        print(generated_samples[0])
        print(generated_samples[1])
        generated_samples = generate_samples_for_digits(sample_size)

    return generated_samples

def print_elapsed_time():
    end_time = int(time.time())
    d = divmod(end_time-start_time,86400)  # days
    h = divmod(d[1],3600)  # hours
    m = divmod(h[1],60)  # minutes
    s = m[1]  # seconds

    print('Elapsed Time: %d days, %d hours, %d minutes, %d seconds' % (d[0],h[0],m[0],s))

def calculate_results_matrices(distances_real_vs_sample,distances_real_vs_train, d_min=0.1):

    results_sample = np.zeros((len(distances_real_vs_sample),4))
    for i in range(len(results_sample)):
        # indicate that dataset is a sample
        results_sample[i][0] = 0
        
        integral_approx = 0
        integral_approx_log = 0
        integral_approx_eps = 0
        for eps in distances_real_vs_sample[i]:
            if eps < d_min:
                integral_approx = integral_approx + d_min/eps
                integral_approx_log = integral_approx_log + (-np.log(eps/d_min))
                integral_approx_eps = integral_approx_eps + 1

        integral_approx = integral_approx/len(distances_real_vs_sample[0])
        integral_approx_log = integral_approx_log/len(distances_real_vs_sample[0])
        integral_approx_eps = integral_approx_eps/len(distances_real_vs_sample[0])

        results_sample[i][1] = integral_approx_log
        results_sample[i][2] = integral_approx_eps
        results_sample[i][3] = integral_approx

    results_train = np.zeros((len(distances_real_vs_train),4))
    for i in range(len(results_train)):
        # indicate that dataset is a training data set
        results_train[i][0] = 1
        
        integral_approx = 0
        integral_approx_log = 0
        integral_approx_eps = 0
        for eps in distances_real_vs_train[i]:
            if eps < d_min:
                integral_approx = integral_approx + d_min/eps
                integral_approx_log = integral_approx_log + (-np.log(eps/d_min))
                integral_approx_eps = integral_approx_eps + 1

        integral_approx = integral_approx/len(distances_real_vs_train[0])
        integral_approx_log = integral_approx_log/len(distances_real_vs_train[0])
        integral_approx_eps = integral_approx_eps/len(distances_real_vs_train[0])

        results_train[i][1] = integral_approx_log
        results_train[i][2] = integral_approx_eps
        results_train[i][3] = integral_approx
        
    return results_sample,results_train

def mc_attack_sample(results_sample, results_train):
    results = np.concatenate((results_sample, results_train))
    np.random.shuffle(results)
    mc_attack_log = results[results[:,1].argsort()][:,0][-len(results_train):].mean()
    np.random.shuffle(results)
    mc_attack_eps = results[results[:,2].argsort()][:,0][-len(results_train):].mean()
    np.random.shuffle(results)
    mc_attack_frac = results[results[:,3].argsort()][:,0][-len(results_train):].mean()

    successfull_set_attack_1 = results_train[:,1].sum() > results_sample[:,1].sum()
    successfull_set_attack_2 = results_train[:,2].sum() > results_sample[:,2].sum()
    successfull_set_attack_3 = results_train[:,3].sum() > results_sample[:,3].sum()

    return mc_attack_log, mc_attack_eps, mc_attack_frac, successfull_set_attack_1, successfull_set_attack_2, successfull_set_attack_3

def mc_attack(results_sample, results_train):

    mc_attack_log, mc_attack_eps, mc_attack_frac, successfull_set_attack_1, successfull_set_attack_2, successfull_set_attack_3 = mc_attack_sample(results_sample, results_train)

    print('50_perc_mc_attack_log: %.3f'%(mc_attack_log))
    print('50_perc_mc_attack_eps: %.3f'%(mc_attack_eps))
    print('50_perc_mc_attack_frac: %.3f'%(mc_attack_frac))
    print('successfull_set_attack_1: %.3f'%(successfull_set_attack_1))
    print('successfull_set_attack_2: %.3f'%(successfull_set_attack_2))
    print('successfull_set_attack_3: %.3f'%(successfull_set_attack_3))

    iterations = 1000
    results_attacks = np.zeros((iterations, 3))

    for i in range(len(results_attacks)):
        np.random.shuffle(results_train)
        res = mc_attack_sample(results_sample, results_train[0:10])
        results_attacks[i][0] = res[0]
        results_attacks[i][1] = res[1]
        results_attacks[i][2] = res[2]

    print('11_perc_mc_attack_log: %.3f'%(results_attacks[:,0].mean()))
    print('11_perc_mc_attack_eps: %.3f'%(results_attacks[:,1].mean()))
    print('11_perc_mc_attack_frac: %.3f'%(results_attacks[:,2].mean()))

    return mc_attack_log, mc_attack_eps, mc_attack_frac, results_attacks[:,0].mean(), results_attacks[:,1].mean(), results_attacks[:,2].mean(), successfull_set_attack_1, successfull_set_attack_2, successfull_set_attack_3

def euclidean_PCA_mc_attack(n_components_pca, trX_inds, vaX_inds, exp_no, mc_euclidean_no_batches, mc_sample_size, percentiles):
    pca = PCA(n_components=n_components_pca)

    pca.fit_transform(teX.reshape((len(teX),784)))

    euclidean_trX = np.reshape(trX, (len(trX),784,))
    euclidean_trX = euclidean_trX[trX_inds]
    euclidean_trX = pca.transform(euclidean_trX)

    euclidean_vaX = np.reshape(vaX, (len(vaX),784,))
    euclidean_vaX = euclidean_vaX[vaX_inds]
    euclidean_vaX = pca.transform(euclidean_vaX)

    distances_trX = np.zeros((len(euclidean_trX), mc_euclidean_no_batches*mc_sample_size))
    distances_vaX = np.zeros((len(euclidean_vaX), mc_euclidean_no_batches*mc_sample_size))

    for i in range(mc_euclidean_no_batches):

        print('Working on %d/%d'%(i, mc_euclidean_no_batches))

        euclidean_generated_samples = generate_samples_for_digits(mc_sample_size)

        #euclidean_generated_samples = euclidean_generated_samples - euclidean_generated_samples.min()
        #euclidean_generated_samples = euclidean_generated_samples*255/euclidean_generated_samples.max()
        euclidean_generated_samples = np.reshape(euclidean_generated_samples, (len(euclidean_generated_samples),784,))
        euclidean_generated_samples = pca.transform(euclidean_generated_samples)

        distances_trX_partial = scipy.spatial.distance.cdist(euclidean_trX, euclidean_generated_samples, 'euclidean')
        distances_vaX_partial = scipy.spatial.distance.cdist(euclidean_vaX, euclidean_generated_samples, 'euclidean')

        # optimized, better than concatenate
        distances_trX[:,i*mc_sample_size:(i+1)*mc_sample_size] = distances_trX_partial
        distances_vaX[:,i*mc_sample_size:(i+1)*mc_sample_size] = distances_vaX_partial
        
        print_elapsed_time()

    for percentile in percentiles:
        print_elapsed_time()
        print('Calculating Results Matrices for '+str(percentile)+' Percentile...')

        d_min = np.percentile(np.concatenate((distances_trX,distances_vaX)),percentile)
        results_sample,results_train = calculate_results_matrices(distances_vaX, distances_trX,d_min)
        
        # save data
        new_row = np.zeros(1, dtype = dt)[0]
        new_row['instance_no'] = instance_no
        new_row['exp_no'] = exp_no
        new_row['method'] = 2 # euclidean PCA
        new_row['pca_n'] = n_components_pca
        new_row['percentage_of_data'] = percentage
        new_row['percentile'] = percentile
        new_row['mc_euclidean_no_batches'] = mc_euclidean_no_batches

        mc_attack_results = mc_attack(results_sample, results_train)
        new_row['50_perc_mc_attack_log'] = mc_attack_results[0]
        new_row['50_perc_mc_attack_eps'] = mc_attack_results[1]
        new_row['50_perc_mc_attack_frac'] = mc_attack_results[2]
        new_row['11_perc_mc_attack_log'] = mc_attack_results[3]
        new_row['11_perc_mc_attack_eps'] = mc_attack_results[4]
        new_row['11_perc_mc_attack_frac'] = mc_attack_results[5]
        
        experiment_results.append(new_row)
        np.savetxt(experiment+'.csv', np.array(experiment_results, dtype = dt))

def euclidean_PCA_mc_attack_category(n_components_pca, trX_inds, vaX_inds, exp_no, mc_euclidean_no_batches, mc_sample_size, percentiles):
    pca = PCA(n_components=n_components_pca)

    pca.fit_transform(teX.reshape((len(teX),784)))

    euclidean_trX = np.reshape(trX, (len(trX),784,))
    euclidean_trX = euclidean_trX[trX_inds]
    euclidean_trX = pca.transform(euclidean_trX)

    euclidean_vaX = np.reshape(vaX, (len(vaX),784,))
    euclidean_vaX = euclidean_vaX[vaX_inds]
    euclidean_vaX = pca.transform(euclidean_vaX)

    distances_trX = np.zeros((len(euclidean_trX), mc_euclidean_no_batches*mc_sample_size // 10))
    distances_vaX = np.zeros((len(euclidean_vaX), mc_euclidean_no_batches*mc_sample_size // 10))

    for i in range(mc_euclidean_no_batches):

        print('Working on %d/%d'%(i, mc_euclidean_no_batches))

        euclidean_generated_samples = generate_samples_for_digits(mc_sample_size)

        euclidean_generated_samples = np.reshape(euclidean_generated_samples, (len(euclidean_generated_samples),784,))
        euclidean_generated_samples = pca.transform(euclidean_generated_samples)
        
        for digit in range(10):
            # indexes of 1's, 2's, 3's etc.
            digit_indexes_train = np.where(trY[trX_inds] == digit)
            digit_indexes_sample = [digit+10*i for i in range(mc_sample_size//10)]
            # only compare to current digit
            distances_trX[digit_indexes_train,i*mc_sample_size//10:(i+1)*mc_sample_size//10] = scipy.spatial.distance.cdist(euclidean_trX[digit_indexes_train], euclidean_generated_samples[digit_indexes_sample], 'euclidean')

        for digit in range(10):
            # indexes of 1's, 2's, 3's etc.
            digit_indexes_va = np.where(vaY[vaX_inds] == digit)
            digit_indexes_sample = [digit+10*i for i in range(mc_sample_size//10)]
            # only compare to current digit
            distances_vaX[digit_indexes_va,i*mc_sample_size//10:(i+1)*mc_sample_size//10] = scipy.spatial.distance.cdist(euclidean_vaX[digit_indexes_va], euclidean_generated_samples[digit_indexes_sample], 'euclidean')
        
        print_elapsed_time()

    for percentile in percentiles:
        print_elapsed_time()
        print('Calculating Results Matrices for '+str(percentile)+' Percentile...')

        d_min = np.percentile(np.concatenate((distances_trX,distances_vaX)),percentile)
        results_sample,results_train = calculate_results_matrices(distances_vaX, distances_trX,d_min)

        # save data
        new_row = np.zeros(1, dtype = dt)[0]
        new_row['instance_no'] = instance_no
        new_row['exp_no'] = exp_no
        new_row['method'] = 4 # euclidean PCA cat
        new_row['pca_n'] = n_components_pca
        new_row['percentage_of_data'] = percentage
        new_row['percentile'] = percentile
        new_row['mc_euclidean_no_batches'] = mc_euclidean_no_batches

        mc_attack_results = mc_attack(results_sample, results_train)
        new_row['50_perc_mc_attack_log'] = mc_attack_results[0]
        new_row['50_perc_mc_attack_eps'] = mc_attack_results[1]
        new_row['50_perc_mc_attack_frac'] = mc_attack_results[2]
        new_row['11_perc_mc_attack_log'] = mc_attack_results[3]
        new_row['11_perc_mc_attack_eps'] = mc_attack_results[4]
        new_row['11_perc_mc_attack_frac'] = mc_attack_results[5]
        
        experiment_results.append(new_row)
        np.savetxt(experiment+'.csv', np.array(experiment_results, dtype = dt))

    print('Calculating Results Matrices for flexible d_min...')
    distances = np.concatenate((distances_trX,distances_vaX))
    d_min = np.median([distances[i].min() for i in range(len(distances))])
    results_sample,results_train = calculate_results_matrices(distances_vaX, distances_trX,d_min)

    # save data
    new_row = np.zeros(1, dtype = dt)[0]
    new_row['instance_no'] = instance_no
    new_row['exp_no'] = exp_no
    new_row['method'] = 4 # euclidean PCA cat
    new_row['pca_n'] = n_components_pca
    new_row['percentage_of_data'] = percentage
    new_row['percentile'] = -1 # dynamic
    new_row['mc_euclidean_no_batches'] = mc_euclidean_no_batches
    mc_attack_results = mc_attack(results_sample, results_train)
    new_row['50_perc_mc_attack_log'] = mc_attack_results[0]
    new_row['50_perc_mc_attack_eps'] = mc_attack_results[1]
    new_row['50_perc_mc_attack_frac'] = mc_attack_results[2]
    new_row['11_perc_mc_attack_log'] = mc_attack_results[3]
    new_row['11_perc_mc_attack_eps'] = mc_attack_results[4]
    new_row['11_perc_mc_attack_frac'] = mc_attack_results[5]
    
    experiment_results.append(new_row)
    np.savetxt(experiment+'.csv', np.array(experiment_results, dtype = dt))

    return results_sample,results_train

def generate_batch_hog_features(samples):
    features_matrix = np.zeros((len(samples),81))

    for i in range(len(samples)):
        features_matrix[i] = hog(samples[i].reshape((28, 28)), orientations=9, pixels_per_cell=(9, 9), visualise=False) #, transform_sqrt=True, block_norm='L2-Hys')
    
    return features_matrix

def hog_mc_attack(trX_inds, vaX_inds, exp_no, mc_hog_no_batches, mc_sample_size, percentiles):

    feature_matrix_vaX = generate_batch_hog_features(vaX[vaX_inds])
    feature_matrix_trX = generate_batch_hog_features(trX[trX_inds])

    distances_trX = np.zeros((len(feature_matrix_trX), mc_hog_no_batches*mc_sample_size))
    distances_vaX = np.zeros((len(feature_matrix_vaX), mc_hog_no_batches*mc_sample_size))

    for i in range(mc_hog_no_batches):

        print('Working on %d/%d'%(i, mc_hog_no_batches))

        generated_samples = generate_samples_for_digits(mc_sample_size)

        generated_samples = generated_samples - generated_samples.min()
        generated_samples = generated_samples*255/generated_samples.max()

        feature_matrix_generated = generate_batch_hog_features(generated_samples)

        distances_trX_partial = scipy.spatial.distance.cdist(feature_matrix_trX, feature_matrix_generated, 'euclidean')
        distances_vaX_partial = scipy.spatial.distance.cdist(feature_matrix_vaX, feature_matrix_generated, 'euclidean')

        # optimized, better than concatenate
        distances_trX[:,i*mc_sample_size:(i+1)*mc_sample_size] = distances_trX_partial
        distances_vaX[:,i*mc_sample_size:(i+1)*mc_sample_size] = distances_vaX_partial

        print_elapsed_time()

    for percentile in percentiles:
        print_elapsed_time()
        print('Calculating Results Matrices for '+str(percentile)+' Percentile...')

        d_min = np.percentile(np.concatenate((distances_trX,distances_vaX)),percentile)
        results_sample,results_train = calculate_results_matrices(distances_vaX, distances_trX,d_min)

        # save data
        new_row = np.zeros(1, dtype = dt)[0]
        new_row['instance_no'] = instance_no
        new_row['exp_no'] = exp_no
        new_row['method'] = 3
        new_row['percentage_of_data'] = percentage
        new_row['percentile'] = percentile
        new_row['mc_hog_no_batches'] = mc_hog_no_batches

        mc_attack_results = mc_attack(results_sample, results_train)
        new_row['50_perc_mc_attack_log'] = mc_attack_results[0]
        new_row['50_perc_mc_attack_eps'] = mc_attack_results[1]
        new_row['50_perc_mc_attack_frac'] = mc_attack_results[2]
        new_row['11_perc_mc_attack_log'] = mc_attack_results[3]
        new_row['11_perc_mc_attack_eps'] = mc_attack_results[4]
        new_row['11_perc_mc_attack_frac'] = mc_attack_results[5]
        
        experiment_results.append(new_row)
        np.savetxt(experiment+'.csv', np.array(experiment_results, dtype = dt))

def hog_mc_attack_category(trX_inds, vaX_inds, exp_no, mc_hog_no_batches, mc_sample_size, percentiles):

    feature_matrix_vaX = generate_batch_hog_features(vaX[vaX_inds])
    feature_matrix_trX = generate_batch_hog_features(trX[trX_inds])

    distances_trX = np.zeros((len(feature_matrix_trX), mc_hog_no_batches*mc_sample_size // 10))
    distances_vaX = np.zeros((len(feature_matrix_vaX), mc_hog_no_batches*mc_sample_size // 10))

    for i in range(mc_hog_no_batches):

        print('Working on %d/%d'%(i, mc_hog_no_batches))

        generated_samples = generate_samples_for_digits(mc_sample_size)

        generated_samples = generated_samples - generated_samples.min()
        generated_samples = generated_samples*255/generated_samples.max()

        feature_matrix_generated = generate_batch_hog_features(generated_samples)

        for digit in range(10):
            # indexes of 1's, 2's, 3's etc.
            digit_indexes_train = np.where(trY[trX_inds] == digit)
            digit_indexes_sample = [digit+10*i for i in range(mc_sample_size//10)]
            # only compare to current digit
            distances_trX[digit_indexes_train,i*mc_sample_size//10:(i+1)*mc_sample_size//10] = scipy.spatial.distance.cdist(feature_matrix_trX[digit_indexes_train], feature_matrix_generated[digit_indexes_sample], 'euclidean')

        for digit in range(10):
            # indexes of 1's, 2's, 3's etc.
            digit_indexes_va = np.where(vaY[vaX_inds] == digit)
            digit_indexes_sample = [digit+10*i for i in range(mc_sample_size//10)]
            # only compare to current digit
            distances_vaX[digit_indexes_va,i*mc_sample_size//10:(i+1)*mc_sample_size//10] = scipy.spatial.distance.cdist(feature_matrix_vaX[digit_indexes_va], feature_matrix_generated[digit_indexes_sample], 'euclidean')

        print_elapsed_time()

    for percentile in percentiles:
        print_elapsed_time()
        print('Calculating Results Matrices for '+str(percentile)+' Percentile...')

        d_min = np.percentile(np.concatenate((distances_trX,distances_vaX)),percentile)
        results_sample,results_train = calculate_results_matrices(distances_vaX, distances_trX,d_min)

        # save data
        new_row = np.zeros(1, dtype = dt)[0]
        new_row['instance_no'] = instance_no
        new_row['exp_no'] = exp_no
        new_row['method'] = 5 # hog cat
        new_row['percentage_of_data'] = percentage
        new_row['percentile'] = percentile
        new_row['mc_hog_no_batches'] = mc_hog_no_batches

        mc_attack_results = mc_attack(results_sample, results_train)
        new_row['50_perc_mc_attack_log'] = mc_attack_results[0]
        new_row['50_perc_mc_attack_eps'] = mc_attack_results[1]
        new_row['50_perc_mc_attack_frac'] = mc_attack_results[2]
        new_row['11_perc_mc_attack_log'] = mc_attack_results[3]
        new_row['11_perc_mc_attack_eps'] = mc_attack_results[4]
        new_row['11_perc_mc_attack_frac'] = mc_attack_results[5]
        
        experiment_results.append(new_row)
        np.savetxt(experiment+'.csv', np.array(experiment_results, dtype = dt))

    print('Calculating Results Matrices for flexible d_min...')
    distances = np.concatenate((distances_trX,distances_vaX))
    d_min = np.median([distances[i].min() for i in range(len(distances))])
    results_sample,results_train = calculate_results_matrices(distances_vaX, distances_trX,d_min)

    # save data
    new_row = np.zeros(1, dtype = dt)[0]
    new_row['instance_no'] = instance_no
    new_row['exp_no'] = exp_no
    new_row['method'] = 5 # hog cat
    new_row['percentage_of_data'] = percentage
    new_row['percentile'] = -1
    new_row['mc_hog_no_batches'] = mc_hog_no_batches

    mc_attack_results = mc_attack(results_sample, results_train)
    new_row['50_perc_mc_attack_log'] = mc_attack_results[0]
    new_row['50_perc_mc_attack_eps'] = mc_attack_results[1]
    new_row['50_perc_mc_attack_frac'] = mc_attack_results[2]
    new_row['11_perc_mc_attack_log'] = mc_attack_results[3]
    new_row['11_perc_mc_attack_eps'] = mc_attack_results[4]
    new_row['11_perc_mc_attack_frac'] = mc_attack_results[5]
    
    experiment_results.append(new_row)
    np.savetxt(experiment+'.csv', np.array(experiment_results, dtype = dt))

    return results_sample,results_train

start_time = int(time.time())

trX_inds = np.arange(len(trX))
np.random.shuffle(trX_inds)
trX_inds = trX_inds[0:100]

vaX_inds = np.arange(len(trX))
np.random.shuffle(vaX_inds)
vaX_inds = vaX_inds[0:100]

results_sample_pca,results_train_pca = euclidean_PCA_mc_attack_category(40, trX_inds, vaX_inds, 1, 3, 1000, [])
results_sample_hog,results_train_hog = hog_mc_attack_category(trX_inds, vaX_inds, 1, 3, 1000, [])

Working on 0/3
Elapsed Time: 0 days, 0 hours, 0 minutes, 1 seconds
Working on 1/3
Elapsed Time: 0 days, 0 hours, 0 minutes, 2 seconds
Working on 2/3
Elapsed Time: 0 days, 0 hours, 0 minutes, 2 seconds
Calculating Results Matrices for flexible d_min...
50_perc_mc_attack_log: 0.490
50_perc_mc_attack_eps: 0.490
50_perc_mc_attack_frac: 0.490
successfull_set_attack_1: 0.000
successfull_set_attack_2: 0.000
successfull_set_attack_3: 0.000
11_perc_mc_attack_log: 0.027
11_perc_mc_attack_eps: 0.036
11_perc_mc_attack_frac: 0.028
Working on 0/3


c:\users\d065042\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Elapsed Time: 0 days, 0 hours, 0 minutes, 3 seconds
Working on 1/3
Elapsed Time: 0 days, 0 hours, 0 minutes, 3 seconds
Working on 2/3
Elapsed Time: 0 days, 0 hours, 0 minutes, 4 seconds
Calculating Results Matrices for flexible d_min...
50_perc_mc_attack_log: 0.520
50_perc_mc_attack_eps: 0.520
50_perc_mc_attack_frac: 0.520
successfull_set_attack_1: 1.000
successfull_set_attack_2: 1.000
successfull_set_attack_3: 1.000
11_perc_mc_attack_log: 0.126
11_perc_mc_attack_eps: 0.140
11_perc_mc_attack_frac: 0.140


In [21]:
pca_normalizer = np.max(np.concatenate((results_sample_pca,results_train_pca)))
hog_normalizer = np.max(np.concatenate((results_sample_hog,results_train_hog)))
results_sample_pca = results_sample_pca/pca_normalizer
results_train_pca = results_train_pca/pca_normalizer
results_sample_hog = results_sample_hog/hog_normalizer
results_train_hog = results_train_hog/hog_normalizer

In [15]:
results_sample_combined = results_sample_pca+results_sample_hog
results_train_combined = results_train_pca+results_train_hog
results_train_combined[:,0]=1
exp_no = 1

In [17]:
# save data
new_row = np.zeros(1, dtype = dt)[0]
new_row['instance_no'] = instance_no
new_row['exp_no'] = exp_no
new_row['method'] = 9 # bagging cat
new_row['percentage_of_data'] = percentage
new_row['percentile'] = -1
new_row['mc_hog_no_batches'] = 0

mc_attack_results = mc_attack(results_sample_combined, results_train_combined)
new_row['50_perc_mc_attack_log'] = mc_attack_results[0]
new_row['50_perc_mc_attack_eps'] = mc_attack_results[1]
new_row['50_perc_mc_attack_frac'] = mc_attack_results[2]
new_row['11_perc_mc_attack_log'] = mc_attack_results[3]
new_row['11_perc_mc_attack_eps'] = mc_attack_results[4]
new_row['11_perc_mc_attack_frac'] = mc_attack_results[5]
    
experiment_results.append(new_row)
np.savetxt(experiment+'.csv', np.array(experiment_results, dtype = dt))

50_perc_mc_attack_log: 0.500
50_perc_mc_attack_eps: 0.510
50_perc_mc_attack_frac: 0.510
successfull_set_attack_1: 0.000
successfull_set_attack_2: 0.000
successfull_set_attack_3: 0.000
11_perc_mc_attack_log: 0.030
11_perc_mc_attack_eps: 0.032
11_perc_mc_attack_frac: 0.032


In [22]:
(np.load('trX_vae.npy')*255)[0]

array([2.55000009e-06, 2.55000009e-06, 2.55000009e-06, 2.55000009e-06,
       2.55000009e-06, 2.55000009e-06, 2.55000009e-06, 2.55000009e-06,
       2.55000009e-06, 2.55000009e-06, 2.55000009e-06, 2.55000009e-06,
       2.55000009e-06, 2.55000009e-06, 2.55000009e-06, 2.55000009e-06,
       2.55000009e-06, 2.55000009e-06, 2.55000009e-06, 2.55000009e-06,
       2.55000009e-06, 2.55000009e-06, 2.55000009e-06, 2.55000009e-06,
       2.55000009e-06, 2.55000009e-06, 2.55000009e-06, 2.55000009e-06,
       2.55000009e-06, 2.55000009e-06, 2.55000009e-06, 2.55000009e-06,
       2.55000009e-06, 2.55000009e-06, 2.55000009e-06, 2.55000009e-06,
       2.55000009e-06, 2.55000009e-06, 2.80697418e-06, 1.61272856e-05,
       2.55000009e-06, 6.48454534e-06, 3.88009270e-04, 3.46722627e-05,
       2.18730667e-04, 5.29061363e-04, 2.16184650e-04, 4.34412126e-04,
       2.17582332e-03, 3.45829688e-03, 2.55000009e-06, 2.55000009e-06,
       2.55000009e-06, 2.55000009e-06, 2.55000009e-06, 2.55000009e-06,
      

In [19]:
np.load('trX_vae.npy')
np.load('trY_vae.npy')

array([0, 1, 2, ..., 7, 8, 9])

In [4]:
class Generator(object):
    def __init__(self, digit):
        self.input_dim = dim_z
        self.output_dim = 784
        self.digit = digit

    def __call__(self, z):
        return self.generate_samples_tf(z, self.digit)

    def generate_samples_tf(self, Z,digit):
        
        sample_size = tf.shape(Z)[0]
        
        one_hot_digits = np.zeros(10,)
        one_hot_digits[digit] = 1
        Y_np_sample = tf.reshape(tf.tile(tf.constant(one_hot_digits, dtype=tf.float32),[sample_size]), [sample_size, 10])
        
        #Z_np_sample = tf.cast(Z_np_sample, tf.float32)
        #Y_np_sample = tf.cast(Y_np_sample, tf.float32)
        generated_samples = vae.decoder(Z, Y_np_sample, dim_img, n_hidden)
        
        # transform to match validation data
        generated_samples = tf.reshape(generated_samples, [tf.shape(generated_samples)[0], 784])

        # cast to int
        #generated_samples = tf.cast(generated_samples, tf.int32)

        return generated_samples

In [5]:
generator = Generator(trY[0])
prior = NormalPrior()
kernel = ParsenDensityEstimator()

sigma = 0.025
model = ais.Model(generator, prior, kernel, sigma, 10000)

num_samples = 1
schedule = ais.get_schedule(num_samples, rad=4)
p2 = model.ais(np.array([trX[0]]), schedule)

print(p2)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
(0.7309, array([-147728.], dtype=float32), array([-147728.], dtype=float32))


In [5]:
def OneHot(X, n=10, negative_class=0.):
    X = np.asarray(X).flatten()
    if n is None:
        n = np.max(X) + 1
    Xoh = np.ones((len(X), n)) * negative_class
    Xoh[np.arange(len(X)), X] = 1.
    return Xoh

In [6]:
y = np.zeros((100,))
y[:] = 1
y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [18]:
Y_Labels = [0,1,2,3,4,5,6,7,8,9]
Z_np_sample_buffer = np.random.randn(len(Y_Labels), dim_z)
Y_Labels = OneHot(Y_Labels)

generated_samples = sess.run(decoded, feed_dict={z_in: Z_np_sample_buffer, fack_id_in: Y_Labels, keep_prob : 1})

In [21]:
generated_samples

array([[4.4577275e-10, 4.8040927e-10, 5.2374305e-10, ..., 2.8561100e-09,
        1.1276368e-09, 2.5707847e-09],
       [4.1918931e-09, 3.5248532e-10, 1.2809082e-09, ..., 4.7559273e-10,
        4.4434092e-09, 8.8464451e-11],
       [4.7324999e-10, 1.4354905e-09, 9.5292008e-10, ..., 3.5966325e-09,
        1.6841941e-09, 2.0631703e-10],
       ...,
       [4.5303254e-11, 2.5053085e-10, 2.6552930e-11, ..., 3.2445518e-11,
        5.5640478e-11, 3.1044525e-11],
       [4.5461968e-09, 2.6213141e-09, 3.0995495e-10, ..., 2.3530047e-09,
        3.5507530e-09, 3.4395060e-09],
       [3.3948438e-11, 2.4918551e-10, 3.1829028e-10, ..., 4.2411821e-10,
        4.1339029e-11, 3.7154035e-10]], dtype=float32)

In [19]:
import scipy

def save_image(data, path):
    img = np.zeros((28, 28, 3))
    img[0:28, 0:28, :] = data.reshape( [28, 28, 1])
    scipy.misc.imsave('./'+path+'.jpg', img)

In [20]:
for i in range(len(Y_Labels)):
    save_image(generated_samples[i],'test'+str(i))

c:\users\d065042\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


In [3]:
# indexes 1,11,21,31,... are ones, 2,12,22 are twos etc.
def generate_samples_for_digits(sample_size=100):
    
    Z_np_sample_buffer = np.random.randn(sample_size, dim_z)
    
    digits = np.zeros((sample_size,)).astype(int)
    for i in range(len(digits)):
        digits[i] = i%10
    Y_np_sample = OneHot( digits)

    generated_samples = sess.run(decoded, feed_dict={z_in: Z_np_sample_buffer, fack_id_in: Y_np_sample, keep_prob : 1})
    return generated_samples

In [10]:
generated_samples = generate_samples_for_digits(10000)

In [65]:
len(generated_samples[1])

784